El objetivo es encotrar la informacion mas posible, y que sea de calidad. De acuerdo a los criterios de PLD
La tabla de clientes es

| Información requerida                   | Información obtenida                              |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 |                                                  |
| nombre cliente o razón social           |    NOMBRES, PRIMER_APELLIDO, SEGUNDO_APELLIDO, RAZON_SOCIAL, NOMBRE_COMERCIAL |
| número cliente                          |    codigo_cliente                                         |
| estatus                                 |      Por movimientos                             |    
| número contrato                         | NUMERO DE CONTRADO ORIGEN: PR_CONTRATOS          |
| género                                  |     SEXO IF PF ELSE N/A |
| fecha_nacimiento o de constitución      |   FECHA_DE_NACIMIENTO                            |
| entidad federativa nacimiento           |                                   |
| país de nacimiento                      |           PAIS_NACIMIENTO                                       |
| nacionalidad                            |    SE OBTIENE DE MG_PAISES|
| profesión                               |    CODIGO_PROFESION                                              |
| calle                                   |                                 |
| número_exterior                         |                                                  |
| número_interior                         |                                                  |
| colonia_urbanización                    |                                                  |
| delegación_municipio                    |          CODIGO_MUNICIPIO                                        |
| ciudad_población                        |                                                  |
| entidad_federativa                      |          CODIGO_DEPARTAMENTO                          |
| código postal                           |                                                  |
| país                                    |       CODIGO_PAIS                         |
| teléfono                                |                                    |
| correo_electrónico                      |                                                  |
| rfc                                     | NUMERO_IDENTIFICACION     |
| curp                                    |       CURP                                           |
| fiel                                    |          FIEL                                        |
| representante legal                     |    REPRESENTANTE_LEGAL                                              |
| tipo de persona                         |   CODIGO_TIPO_IDENTIFICACION       5:PF 7:PM   |
| clasificación por grado de riesgo       |                                                  |
| pep (persona políticamente expuesta)    |                                                  |
| razón                                   |                                                  |
| producto contratado                     |                   |
| cambios en perfil transaccional         |          INFORMACION NO ENCONTRADA                                        |